In [ ]:
# 練習4-1 利用公式計算以下BGR轉灰階
import cv2
import numpy as np

# 輸入 B G R (三個一組，可以輸入多組)
a = list(map(int, input("請輸入 B G R 三個數字 (可多組): ").split()))

my_ans = []
for i in range(0, len(a), 3):
    B = a[i]
    G = a[i+1]
    R = a[i+2]

    gray = round(0.299*R + 0.587*G + 0.114*B)
    my_ans.append(gray)

# OpenCV 計算
arr = np.array(a, dtype=np.uint8).reshape(-1, 1, 3)  # (N,1,3)
gray_cv = cv2.cvtColor(arr, cv2.COLOR_BGR2GRAY).reshape(-1)

print("公式計算結果：", my_ans)
print("OpenCV 結果  ：", gray_cv.tolist())



In [ ]:
# 練習4-3 BGR與RGB互轉-實作圖
import cv2
import numpy as np
DTM=cv2.imread('C:/img/DTM.jpg')
b,g,r=cv2.split(DTM)
rgb=cv2.merge([r,g,b])
bgr=rgb[:,:,::-1]
cv2.imshow("DTM",DTM)
cv2.imshow("rgb",rgb)
cv2.imshow("bgr",bgr)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# 練習4-4 利用公式計算以下BGR轉YCrCb
import cv2
import numpy as np

# 輸入 BGR (三個一組)
a = list(map(int, input("請輸入 B G R 三個數字 (可多組): ").split()))

ans = []
for i in range(0, len(a), 3):
    B = a[i]
    G = a[i+1]
    R = a[i+2]

    # YCrCb 公式 (BT.601)
    Y  = round(0.299*R + 0.587*G + 0.114*B)
    Cr = round((R - Y) * 0.713 + 128)
    Cb = round((B - Y) * 0.564 + 128)


    ans.append([Y, Cr, Cb])

# OpenCV 轉換
arr = np.array(a, dtype=np.uint8).reshape(-1, 1, 3)  # 轉成影像格式 (N,1,3)
ycrcb = cv2.cvtColor(arr, cv2.COLOR_BGR2YCrCb).reshape(-1, 3)

print("公式計算結果：", ans)
print("OpenCV 結果  ：", ycrcb.tolist())


In [ ]:
# 練習4-5
import cv2
import numpy as np

def bgr_to_hsv_manual(b, g, r):

    # 1. 正規化到 0~1
    R = r / 255.0
    G = g / 255.0
    B = b / 255.0

    Cmax = max(R, G, B)
    Cmin = min(R, G, B)
    delta = Cmax - Cmin

    # Hue 計算
    if delta == 0:
        H = 0
    elif Cmax == R:
        H = 60 * (((G - B) / delta) % 6)
    elif Cmax == G:
        H = 60 * (((B - R) / delta) + 2)
    elif Cmax == B:
        H = 60 * (((R - G) / delta) + 4)

    # Saturation
    S = 0 if Cmax == 0 else delta / Cmax

    # Value
    V = Cmax

    # 轉成 OpenCV 範圍
    H_cv = int(H / 2)       # 0~179
    S_cv = int(S * 255)     # 0~255
    V_cv = int(V * 255)     # 0~255

    return H_cv, S_cv, V_cv

# BGR 值
bgr_val = (85, 128, 50)
b, g, r = bgr_val

# 手動公式計算
h_manual, s_manual, v_manual = bgr_to_hsv_manual(b, g, r)
print("手動公式 HSV (OpenCV 範圍):", (h_manual, s_manual, v_manual))

# OpenCV 函式
bgr_np = np.uint8([[[b, g, r]]])
hsv_cv = cv2.cvtColor(bgr_np, cv2.COLOR_BGR2HSV)
print("OpenCV 函式 HSV:", hsv_cv[0,0])


In [ ]:
# 練習4-6
import cv2
import numpy as np
from itertools import combinations

# 讀取圖片
files = ["C:/img/test01.png", "C:/img/test02.png", "C:/img/test03.png"]
images = [cv2.imread(f) for f in files]

# 1️⃣ 計算平均顏色
avg_bgr = []
avg_hsv = []
avg_ycrcb = []

for img in images:
    # BGR 平均
    avg_bgr.append(np.mean(img.reshape(-1,3), axis=0))
    
    # HSV 平均
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    avg_hsv.append(np.mean(hsv.reshape(-1,3), axis=0))
    
    # YCrCb 平均
    ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    avg_ycrcb.append(np.mean(ycrcb.reshape(-1,3), axis=0))

# 2️⃣ 計算兩兩間歐幾里得距離
def euclidean_dist(a, b):
    return np.linalg.norm(a-b)

pairs = list(combinations(range(3), 2))  # 兩兩組合
print("=== HSV ===")
i=0
for e in avg_hsv:
    print(f"{files[i]}的H,S,V:",e)
    i+=1
print("\n=== HSV 距離 ===")
for i,j in pairs:
    print(f"{files[i]} - {files[j]}: {euclidean_dist(avg_hsv[i], avg_hsv[j]):.2f}")
print("\n=== BGR ===")
i=0
for e in avg_bgr:
    print(f"{files[i]}的B,G,R:",e)
    i+=1
print("\n=== BGR 距離 ===")
for i,j in pairs:
    print(f"{files[i]} - {files[j]}: {euclidean_dist(avg_bgr[i], avg_bgr[j]):.2f}")
print("\n=== YCrCb ===")
i=0
for e in avg_ycrcb:
    print(f"{files[i]}的Y,Cr,Cb:",e)
    i+=1
print("\n=== YCrCb 距離 ===")
for i,j in pairs:
    print(f"{files[i]} - {files[j]}: {euclidean_dist(avg_ycrcb[i], avg_ycrcb[j]):.2f}")


In [ ]:
# 練習4-7 標記像素範圍產生遮罩
import cv2
import numpy as np
img=np.random.randint(0,256,size=[5,5],dtype=np.uint8)
mini=100
maxi=(200)
mask=cv2.inRange(img,mini,maxi)
new=cv2.bitwise_and(img,img,mask=mask)
print("img=\n",img)
print("mask=\n",mask)
print("new=\n",new)

In [ ]:
# 練習4-8 Application: 顯示ROI
import cv2
import numpy as np

# 1️⃣ 讀取原圖
img = cv2.imread(r"C:\img\opencv.jpg")  # 注意路徑使用 r"" 或雙反斜線

# 2️⃣ 將 BGR 轉 HSV
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# 3️⃣ 設定顏色上下界 (以 BGR 中的一個像素為例)

H=0

# 設定上下界 
lower = np.array([(H-10), 100, 100])
upper = np.array([(H+10), 255, 255])

# 4️⃣ 產生遮罩
mask = cv2.inRange(hsv, lower, upper)

# 5️⃣ 利用遮罩套用在原圖
result = cv2.bitwise_and(img, img, mask=mask)

# 6️⃣ 顯示結果
cv2.imshow("Original", img)
cv2.imshow("rad", result)
H=60
# 設定上下界 
lower = np.array([(H-10), 100, 100])
upper = np.array([(H+10), 255, 255])

# 4️⃣ 產生遮罩
mask = cv2.inRange(hsv, lower, upper)

# 5️⃣ 利用遮罩套用在原圖
result = cv2.bitwise_and(img, img, mask=mask)
cv2.imshow("green", result)
H=120
# 設定上下界 
lower = np.array([(H-10), 100, 100])
upper = np.array([(H+10), 255, 255])

# 4️⃣ 產生遮罩
mask = cv2.inRange(hsv, lower, upper)

# 5️⃣ 利用遮罩套用在原圖
result = cv2.bitwise_and(img, img, mask=mask)
cv2.imshow("blue", result)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# 練習4-9 標記膚色
import cv2
import numpy as np

# 讀取圖片
img = cv2.imread(r"C:\img\lesson2.jpg") 

# 轉 HSV 色彩空間
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# 膚色的 HSV 範圍（可調整）
lower_skin = np.array([5, 0, 0], dtype=np.uint8)
upper_skin = np.array([170, 255, 255], dtype=np.uint8)

# 建立遮罩：膚色 → 白(255)，非膚色 → 黑(0)
mask = cv2.inRange(hsv, lower_skin, upper_skin)

# 顯示結果
cv2.imshow("Original", img)
cv2.imshow("H", mask)
# 膚色的 HSV 範圍（可調整）
lower_skin = np.array([0, 25, 0], dtype=np.uint8)
upper_skin = np.array([255, 166, 255], dtype=np.uint8)

# 建立遮罩：膚色 → 白(255)，非膚色 → 黑(0)
mask2 = cv2.inRange(hsv, lower_skin, upper_skin)
mask3=cv2.bitwise_and(mask,mask2)
# 顯示結果
cv2.imshow("S", mask2)
cv2.imshow("H+S", mask3)
img2 = cv2.bitwise_and(img, img, mask=mask3)
cv2.imshow("img2", img2)
cv2.waitKey(0)
cv2.destroyAllWindows()
